In [76]:
import util
from importlib import reload
import dataset as d
import model as m
import loss as l
reload(util)
reload(l)
reload(d)
reload(m)

de = util.DataExtractor()
gt_df = de.croppedData()

ds = d.CroppedDummyset(gt_df)
m1 = m.YOLOResNet()


Saving to: c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\ds2_dense\ds2_dense\gt_space.json


In [63]:
reload(l)
dd = d.CroppedDummyset(gt_df)

l.mean_average_precision(dd, gt_choice="quarter", pred_choice="one")         # ≈ 0.25


mAP@0.5: 0.3809 (GT='quarter', Pred='one', 3 batch(es))


0.38094186782836914

In [57]:
l.mean_average_precision(dd, gt_choice="checker", pred_choice="one", iou_thresh=0.9)         # ≈ 0.50


mAP@0.9: 0.5456 (GT='checker', Pred='one', 3 batch(es))


0.5455883741378784

In [58]:
reload(l)
l.mean_average_precision(dd, gt_choice="vertical_half", pred_choice="one", iou_thresh=0.9)   # ≈ 0.50

mAP@0.9: 0.6643 (GT='vertical_half', Pred='one', 3 batch(es))


0.6642528772354126

In [59]:
reload(l)
l.mean_average_precision(dd, gt_choice="inverse_half", pred_choice="half", iou_thresh=0.9)   # 0.00  (if boxes tiny)

mAP@0.9: 0.0000 (GT='inverse_half', Pred='half', 3 batch(es))


0.0

## Test unit_precision

In [ ]:
import dataset as d
import loss as l
reload(d)
reload(l)

dd = d.CroppedDummyset(gt_df)

quarter = dd.dummyTensor("quarter")
print("quarter cells with obj > 0.5:",
      (quarter[..., 4] > 0.5).sum().item())   # should be N*N//4 * A

one          = dd.dummyTensor("one")
quarter       = dd.dummyTensor("quarter")
checker       = dd.dummyTensor("checker")
vhalf         = dd.dummyTensor("vertical_half")
half          = dd.dummyTensor("half")
inverse_half  = dd.dummyTensor("inverse_half")


quarter cells with obj > 0.5: 800
{'TP': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [75]:
import torch
from typing import Dict, List

def _iou(box_a: torch.Tensor, box_b: torch.Tensor) -> float:
    """
    IoU between two boxes (cx, cy, w, h), all normalised 0-1.
    Returns a Python float (not a tensor) for simplicity.
    """
    xa1, ya1 = box_a[0] - box_a[2] / 2, box_a[1] - box_a[3] / 2
    xa2, ya2 = box_a[0] + box_a[2] / 2, box_a[1] + box_a[3] / 2
    xb1, yb1 = box_b[0] - box_b[2] / 2, box_b[1] - box_b[3] / 2
    xb2, yb2 = box_b[0] + box_b[2] / 2, box_b[1] + box_b[3] / 2

    inter_w  = max(0.0, min(xa2, xb2) - max(xa1, xb1))
    inter_h  = max(0.0, min(ya2, yb2) - max(ya1, yb1))
    inter    = inter_w * inter_h
    union    = box_a[2] * box_a[3] + box_b[2] * box_b[3] - inter
    return 0.0 if union == 0 else inter / union


def unit_precision(
        pred: torch.Tensor,
        gt:   torch.Tensor,
        iou_thresh:   float = 0.5,
        score_thresh: float = 0.5,
        return_stats: bool  = False
) -> float | Dict[str, List]:
    """
    Precision = TP / (TP + FP)
    pred: (N,6)  – cx,cy,w,h,cls,score
    gt:   (M,5)  – cx,cy,w,h,cls
    All coords in same space already.

    If return_stats=True → dict with TP / FP arrays per class.
    Otherwise → single precision float.
    """
    # --- Filter low-score predictions ---------------------------
    keep = pred[:, 5] > score_thresh
    pred = pred[keep]

    # Edge cases
    if len(gt) == 0 and len(pred) == 0:
        return 1.0
    if len(pred) == 0:
        return 0.0

    # --- Build per-class lookup --------------------------------
    classes = torch.unique(torch.cat([pred[:, 4], gt[:, 4]])).tolist()
    tp_total, fp_total = 0, 0

    # containers only if needed
    TP_per, FP_per, Conf_per, total_gt_per = None, None, None, None
    if return_stats:
        C = int(max(classes) + 1)
        TP_per   = [[] for _ in range(C)]
        FP_per   = [[] for _ in range(C)]
        Conf_per = [[] for _ in range(C)]
        total_gt_per = [0] * C
        for c in classes:
            total_gt_per[int(c)] = int((gt[:, 4] == c).sum().item())

    # --- Loop per class ----------------------------------------
    for cls in classes:
        cls = int(cls)
        # predictions for this class, sorted by confidence desc
        p_mask  = pred[:, 4] == cls
        p_boxes = pred[p_mask][:, :4]
        p_scores= pred[p_mask][:, 5]
        order   = torch.argsort(p_scores, descending=True)
        p_boxes = p_boxes[order]
        p_scores= p_scores[order]

        # ground-truth for this class
        g_mask  = gt[:, 4] == cls
        g_boxes = gt[g_mask][:, :4]
        matched = [False] * len(g_boxes)

        # --- match preds to GT ---------------------------------
        for k in range(len(p_boxes)):
            box = p_boxes[k]
            best_iou, best_j = 0.0, -1
            for j in range(len(g_boxes)):
                if matched[j]:
                    continue
                iou = _iou(box, g_boxes[j])
                if iou > best_iou:
                    best_iou, best_j = iou, j

            if best_iou >= iou_thresh and best_j >= 0:
                # True-positive
                tp_total += 1
                matched[best_j] = True
                if return_stats:
                    TP_per[cls].append(1)
                    FP_per[cls].append(0)
            else:
                # False-positive
                fp_total += 1
                if return_stats:
                    TP_per[cls].append(0)
                    FP_per[cls].append(1)

            if return_stats:
                Conf_per[cls].append(p_scores[k].item())

    if return_stats:
        return {
            "TP": TP_per,
            "FP": FP_per,
            "Conf": Conf_per,
            "total_gt": total_gt_per
        }

    return 0.0 if (tp_total + fp_total) == 0 else tp_total / (tp_total + fp_total)



# One perfect prediction
pred = torch.tensor([[0.5, 0.5, 0.2, 0.2, 0, 0.9]])
gt   = torch.tensor([[0.5, 0.5, 0.2, 0.2, 0]])
print(unit_precision(pred, gt))              # → 1.0

# Same prediction but wrong class
pred[0, 4] = 1
print(unit_precision(pred, gt))              # → 0.0

# Two predictions, one matches, one FP
pred = torch.tensor([
    [0.5, 0.5, 0.2, 0.2, 0, 0.9],
    [0.2, 0.2, 0.2, 0.2, 0, 0.8]
])
print(unit_precision(pred, gt))  

1.0
0.0
0.5


In [92]:
import eval 
import dataset 
from importlib import reload

reload(eval)
reload(dataset)

d = dataset.CroppedDummyset(gt_df)

one = d.dummyTensor("one")
half = d.dummyTensor("half")
iou = eval.pred_to_iou(one, half)

oh = eval.unit_precision(one, half, iou)
ho = eval.unit_precision(half, one, iou)
oo = eval.unit_precision(one, one, iou)

In [93]:
oh, ho, oo

(1.0, 1.0, 1.0)

In [119]:
import sys, importlib

# 1) Verify which file you're importing
import eval
print("FILE:", eval.__file__)

# 2) Hard-reload
sys.modules.pop('eval', None)
eval = importlib.import_module('eval')

# 3) Make sure runTests calls debug
eval.debug_pred_to_iou()   # should print stuff
eval.runTests(gt_df)

FILE: c:\Users\alexh\Desktop\cv2\obb_anns_hausarbeit\eval.py
max |bp-bt|: 0.0
IoU diag mean/min/max: 1.0 1.0 1.0
oh, ho, oo = 1.0 0.9999967813491821 1.0
conf≥thr: 3200 pc==tc: 3200 tp_m: 3200 iou>0: 3200
conf≥thr: 1600 pc==tc: 3200 tp_m: 1600 iou>0: 3200
